# Bus Tracker

Adam Beigel

24 February 2024

In [4]:
import requests
import os
import json
from dotenv import load_dotenv

### Load API keys

In [5]:
load_dotenv()
bus_api_key = os.getenv('BUS_API_KEY')
train_api_key = os.getenv('TRAIN_API_KEY')

## Make Requests

### Get Stops
Note: the API docs show direction with a space and in the url as `North%20Bound` but that was changed and now it is just one word

In [ ]:
dir_abbrev = {
    'nb': 'Northbound',
    'sb': 'Southbound',
    'eb': 'Eastbound',
    'wb': 'Westbound',
}

def get_stops(route, direction, write_file=False):
    url = 'http://ctabustracker.com/bustime/api/v2/getstops'
    params = {
        'key': bus_api_key,
        'rt': route,
        'dir': dir_abbrev[direction],
        'format': 'json'
    }
    r = requests.get(url, params=params)

    if not write_file:
        return r.json()
    else:
        with open(f'data/bus_stops/{route}{direction}.json', 'w') as outfile:
            outfile.write(json.dumps(r.json(), indent=4))

In [ ]:
get_stops(36, 'nb', write_file=True)

### Get Predictions

In [17]:
def get_predictions(stop_id):
    url = 'http://www.ctabustracker.com/bustime/api/v2/getpredictions'
    params = {
        'key': bus_api_key,
        'stpid': stop_id,
        'format': 'json'
    }
    r = requests.get(url, params=params)

    # TODO: error handling
    return r.json()['bustime-response']['prd']

In [62]:
preds = get_predictions(5960)
preds[0]

{'tmstmp': '20240224 01:14',
 'typ': 'A',
 'stpnm': 'Halsted & Wrightwood',
 'stpid': '5960',
 'vid': '7943',
 'dstp': 17009,
 'rt': '8',
 'rtdd': '8',
 'rtdir': 'Northbound',
 'des': 'Waveland/Broadway',
 'prdtm': '20240224 01:30',
 'tablockid': '8 -708',
 'tatripid': '1078389',
 'origtatripno': '251983103',
 'dly': False,
 'prdctdn': '15',
 'zone': ''}

In [63]:
print(f"The time is {preds[0]['tmstmp'][9:]}")
print(f"{preds[0]['rt']} {preds[0]['rtdir']}: {preds[0]['stpnm']}")
for p in preds:
    print(f"   {p['prdctdn']} min ({p['prdtm'][9:]})")

The time is 01:14
8 Northbound: Halsted & Wrightwood
   15 min (01:30)
